Import Packages

In [1]:
import time
import random
from collections import deque, namedtuple

import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

from kaggle_environments import evaluate, make, utils

2022-11-15 14:42:48.739990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 14:42:49.013182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-15 14:42:49.013231: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-15 14:42:49.080525: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 14:42:51.309905: W tensorflow/stream_executor/platform/de

Loading environment lux_ai_2022 failed: No module named 'vec_noise'


Hyperparameters

In [ ]:
# Set the random seed for TensorFlow
tf.random.set_seed(0)
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

ConnectX - Kaggle Environment

In [ ]:
env = make("connectx", debug=True)
env.render()
print(env.name, env.version)
print("Default Agents: ", *env.agents)
print('env: ', env.specification.observation)

In [ ]:
state_size = (env.specification.configuration.rows.default, env.specification.configuration.columns.default)
num_actions = env.specification.configuration.columns.default

print('State Shape:', state_size)
print('Number of actions:', num_actions)

Deep Q-Learning

In [ ]:
# Create the Q-Network
q_network = Sequential([
    Input(shape=state_size),                      
    Dense(units=21, activation='relu'),            
    Dense(units=14, activation='relu'),            
    Dense(units=num_actions, activation='linear'),
])

# Create the target Q^-Network
target_q_network = Sequential([
    Input(shape=state_size),                      
    Dense(units=21, activation='relu'),            
    Dense(units=14, activation='relu'),            
    Dense(units=num_actions, activation='linear'),
])

optimizer = Adam(learning_rate=ALPHA)

Experience Replay

In [ ]:
# Store experiences as named tuples
experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

Epsilon-greedy action function

In [ ]:
def get_action(q_values, epsilon=0):
    if random.random() > epsilon:
        return np.argmax(q_values.numpy()[0])
    else:
        return random.choice(np.arange(num_actions))